In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **데이터 살펴보기**

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [3]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/월간 09/Writing_Style/Writing_Style')

FileNotFoundError: ignored

In [5]:
#파일 불러오기
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')

In [6]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [7]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [9]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# **전처리**

In [10]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [11]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [12]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as",
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could",
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has",
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself",
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself",
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours",
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that",
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll",
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll",
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom",
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [13]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [14]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])


In [15]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')

# **모델링**

In [16]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [17]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [18]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [19]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [20]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 16)           320000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 5)                 125       
                                                                 
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train,
                    epochs=num_epochs, verbose=2,
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 11s - loss: 1.5615 - accuracy: 0.2780 - val_loss: 1.5266 - val_accuracy: 0.2712 - 11s/epoch - 8ms/step
Epoch 2/20
1372/1372 - 10s - loss: 1.4082 - accuracy: 0.4140 - val_loss: 1.2782 - val_accuracy: 0.5159 - 10s/epoch - 7ms/step
Epoch 3/20
1372/1372 - 10s - loss: 1.1571 - accuracy: 0.5484 - val_loss: 1.0934 - val_accuracy: 0.5788 - 10s/epoch - 7ms/step
Epoch 4/20
1372/1372 - 9s - loss: 0.9976 - accuracy: 0.6129 - val_loss: 0.9778 - val_accuracy: 0.6251 - 9s/epoch - 7ms/step
Epoch 5/20
1372/1372 - 10s - loss: 0.8911 - accuracy: 0.6610 - val_loss: 0.9093 - val_accuracy: 0.6593 - 10s/epoch - 8ms/step
Epoch 6/20
1372/1372 - 10s - loss: 0.8136 - accuracy: 0.6936 - val_loss: 0.8814 - val_accuracy: 0.6727 - 10s/epoch - 8ms/step
Epoch 7/20
1372/1372 - 9s - loss: 0.7544 - accuracy: 0.7187 - val_loss: 0.8583 - val_accuracy: 0.6680 - 9s/epoch - 7ms/step
Epoch 8/20
1372/1372 - 10s - loss: 0.7071 - accuracy: 0.7379 - val_loss: 0.8180 - val_accuracy: 0.6882 - 10s/epoch - 8ms/s

In [23]:
# predict values
pred = model.predict(test_padded)

614/614 [==============================] - 1s 2ms/step


In [24]:
pred

array([[4.8891761e-06, 9.1960895e-01, 3.5305053e-02, 4.5061123e-02,
        1.9944309e-05],
       [1.4207165e-01, 6.5472734e-01, 6.8060383e-02, 9.8454818e-02,
        3.6685847e-02],
       [9.9936062e-01, 6.4148007e-05, 1.3644619e-07, 8.6739948e-07,
        5.7438796e-04],
       ...,
       [8.1082428e-05, 9.9990052e-01, 5.5279621e-08, 1.8188903e-05,
        7.0380530e-08],
       [8.2039376e-05, 9.9990898e-01, 3.9221760e-07, 8.0573709e-06,
        4.8624287e-07],
       [9.9453580e-01, 7.4237505e-07, 1.5328800e-06, 2.1456949e-06,
        5.4598413e-03]], dtype=float32)

In [25]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,4.889176e-06,9.196090e-01,3.530505e-02,4.506112e-02,1.994431e-05
1,1,1.420716e-01,6.547273e-01,6.806038e-02,9.845482e-02,3.668585e-02
2,2,9.993606e-01,6.414801e-05,1.364462e-07,8.673995e-07,5.743880e-04
3,3,1.496766e-05,1.845838e-08,7.582405e-01,1.484842e-05,2.417296e-01
4,4,8.532497e-01,1.472397e-03,1.457542e-04,1.440707e-01,1.061409e-03
...,...,...,...,...,...,...
19612,19612,7.572277e-08,9.999998e-01,3.712089e-17,3.657712e-09,2.852583e-16
19613,19613,1.009538e-02,9.883134e-07,1.062268e-04,1.104672e-08,9.897974e-01
19614,19614,8.108243e-05,9.999005e-01,5.527962e-08,1.818890e-05,7.038053e-08
19615,19615,8.203938e-05,9.999090e-01,3.922176e-07,8.057371e-06,4.862429e-07


In [26]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')